###  Created by Luis Alejandro (alejand@umich.edu)

In [1]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
import tensorflow as tf
from tfrecord_dataset import feature_description
from aircraft_models import AirMultinomialRNN

In [2]:
# Constants
AUTOTUNE = tf.data.experimental.AUTOTUNE
BUFFER_SIZE = 1000
BATCH_SIZE = 64

In [3]:
# Dataset location
train_file = 'exports/2020-03-01 07-34-19/train.tfrecord'
test_file = 'exports/2020-03-01 07-34-19/test.tfrecord'

In [4]:
# Parses observation from proto format
def tf_parse_observation(example):
    observation = tf.io.parse_single_example(example, feature_description)
    mfcc = observation['mfcc']
    samples = observation['samples']
    observation['spec'] = tf.reshape(observation['spec'],(mfcc,samples))
    return observation

# Converts into correct format for training (input,output) = (spec,label)
def tf_transform_observation(data):
    classes = [b'A320-2xx (CFM56-5)', b'B737-7xx (CF56-7B22-)', b'ERJ190 (CF34-10E)', b'B737-8xx (CF56-7B22+)',
               b'ERJ145 (AE3007)', b'A320-2xx (V25xx)', b'A319-1xx (V25xx)', b'ERJ170-175 (CF34-8E)']
    output = 0
    for i, label in enumerate(classes):
        if data['label'] == label:
            output = i
    return tf.expand_dims(data['spec'],-1), output

In [5]:
# Creates training data pipeline
train_dataset = tf.data.TFRecordDataset([train_file])
train_dataset = train_dataset.map(tf_parse_observation,num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(tf_transform_observation,num_parallel_calls=AUTOTUNE).cache()
observation = next(iter(train_dataset))
mfcc, time, channel = observation[0].shape
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(1)

In [6]:
# Creates test data pipeline
test_dataset = tf.data.TFRecordDataset([test_file])
test_dataset = test_dataset.map(tf_parse_observation,num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.map(tf_transform_observation,num_parallel_calls=AUTOTUNE).cache()
test_dataset = test_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(1)

In [7]:
# Creates model
builder = AirMultinomialRNN(categories=8, use_sequencer=True)
model = builder.build_model(input_shape=[mfcc, time, channel])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 401, 1)]     0         
_________________________________________________________________
Sequencer (SpecSequence)     (None, 15, 128, 50, 1)    0         
_________________________________________________________________
Conv1 (TimeDistributed)      (None, 15, 124, 46, 32)   832       
_________________________________________________________________
Pool1 (TimeDistributed)      (None, 15, 41, 15, 32)    0         
_________________________________________________________________
Conv2 (TimeDistributed)      (None, 15, 37, 11, 32)    25632     
_________________________________________________________________
Pool2 (TimeDistributed)      (None, 15, 12, 3, 32)     0         
_________________________________________________________________
Flatten (TimeDistributed)    (None, 15, 1152)          0     

In [8]:
# Training model
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
model.fit(train_dataset, epochs=60, validation_data=test_dataset)

Epoch 1/60
20/20 [==============================] - 11s 531ms/step - loss: 4.3433 - accuracy: 0.3045 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/60
20/20 [==============================] - 2s 123ms/step - loss: 3.5268 - accuracy: 0.5064 - val_loss: 3.2397 - val_accuracy: 0.5200
Epoch 3/60
20/20 [==============================] - 2s 121ms/step - loss: 2.9388 - accuracy: 0.6146 - val_loss: 2.7993 - val_accuracy: 0.5800
Epoch 4/60
20/20 [==============================] - 2s 122ms/step - loss: 2.5003 - accuracy: 0.6971 - val_loss: 2.4526 - val_accuracy: 0.5600
Epoch 5/60
20/20 [==============================] - 2s 124ms/step - loss: 2.1619 - accuracy: 0.7067 - val_loss: 2.1007 - val_accuracy: 0.6800
Epoch 6/60
20/20 [==============================] - 2s 122ms/step - loss: 1.8604 - accuracy: 0.7524 - val_loss: 1.8980 - val_accuracy: 0.6000
Epoch 7/60
20/20 [==============================] - 2s 123ms/step - loss: 1.6206 - accuracy: 0.7877 - val_loss: 1.7005 - val_accuracy: 0.74

Epoch 59/60
20/20 [==============================] - 3s 128ms/step - loss: 0.5674 - accuracy: 1.0000 - val_loss: 0.9406 - val_accuracy: 0.8400
Epoch 60/60
20/20 [==============================] - 2s 125ms/step - loss: 0.5696 - accuracy: 1.0000 - val_loss: 1.0181 - val_accuracy: 0.8200
